# import libraries

In [5]:
!pip install cv2

ERROR: Could not find a version that satisfies the requirement cv2 (from versions: none)
ERROR: No matching distribution found for cv2


In [4]:
import pandas as pd
import numpy as np
import cv2
import glob
import os
import re

from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB,GaussianNB,BernoulliNB

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

from nltk.stem import PorterStemmer,WordNetLemmatizer
from nltk.tokenize import word_tokenize

ModuleNotFoundError: No module named 'cv2'

In [ ]:
scanned pdf:
    1. pdf2image

readable pdf:
    1. pdftotext
    2. tabula-py (tabular data from pdf)
    3. camelot ((tabular data from pdf))
    4. PyPdf2
    
image to text:
    Image processing steps:
        1. Skew correction by using openCV ()
        2. Noise removal
        
    OCR:
        1. Pytesseract(open source)
        2. google vision
        3. amazon textract

In [4]:
import os
os.getcwd()

'D:\\RahulK\\Velocity_Sep_21\\01_04'

In [11]:
folder_path = "movie_reviews\\"

In [12]:
folders = os.listdir(folder_path)
folders

['negative', 'positive']

In [15]:
pos_review_path = folder_path + "positive"
pos_review_path
neg_review_path = folder_path + "negative"
neg_review_path

'movie_reviews\\negative'

In [33]:
pos_review_txt_files = glob.glob(f"{pos_review_path}\\*.txt")
neg_review_txt_files = glob.glob(f"{neg_review_path}\\*.txt")
# pos_review_txt_files # list of text files 
len(neg_review_txt_files)

1000

In [131]:
def stem_words(text):
    ps = PorterStemmer()
    text_list = []
    tokens = word_tokenize(text)
    for word in tokens:
        word = ps.stem(word)
        text_list.append(word)
    text = ' '.join(text_list)
    print(text)
    return text

from nltk.corpus import wordnet
from nltk import pos_tag

def get_pos_tag(word):
    tag = pos_tag([word])[0][1][0].lower()
    return tag


def lemmatize_text(text):
    lemma = WordNetLemmatizer()
    text_list = []
    tokens = word_tokenize(text)
    for word in tokens:
        print(word)
        word = lemma.lemmatize(word,pos = get_pos_tag(word))
        text_list.append(word)
    text = ' '.join(text_list)
    print(text)
    return text

In [132]:


review_list = []
for file_name in pos_review_txt_files:
    f = open(file_name,'r')
    text = f.read()
    text = re.sub('[^A-Za-z]+', ' ',text)
    text = stem_words(text)
#     text = lemmatize_text(text)
    f.close()
    review_list.append(text)
    
for file_name in neg_review_txt_files:
    f = open(file_name,'r')
    text = f.read()
    text = text.lower()
    text = re.sub('[^A-Za-z]+', ' ',text)
    f.close()
    review_list.append(text)
    
len(review_list)

films
adapted
from


KeyError: 'i'

# Target Variable

In [53]:
pos_target = np.ones(len(pos_review_txt_files),dtype = int)
neg_target = np.zeros(len(neg_review_txt_files),dtype = int)

In [60]:
array = np.full(10,2)
array

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [62]:
y = np.append(pos_target,neg_target)
y = pd.Series(y)
y

0       1
1       1
2       1
3       1
4       1
       ..
1995    0
1996    0
1997    0
1998    0
1999    0
Length: 2000, dtype: int32

# Bag of words

In [88]:
count_vector = CountVectorizer(lowercase=True, stop_words='english', min_df = 10)
x_count = count_vector.fit_transform(review_list)
column_names = count_vector.get_feature_names()
x_cnt_vect = pd.DataFrame(x_count.toarray(),columns=column_names)
x_cnt_vect

# min_df = 5 >> It ignores the term that appears in less than 5 documents
# min_df = 0.05 >> It ignores the term that appears in less than 5% documents

,aaron,abandon,abandoned,abilities,ability,able,ably,aboard,abound,abruptly,...,youth,yup,zane,zany,zellweger,zero,zeta,zombie,zone,zooms
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1996,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1997,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# TF-IDF Vectorizer

In [89]:
tfidf_vector = TfidfVectorizer(lowercase=True, stop_words='english', min_df = 10)
x_tfidf = tfidf_vector.fit_transform(review_list)
column_names = tfidf_vector.get_feature_names()
x_tfidf_vect = pd.DataFrame(x_tfidf.toarray(),columns=column_names)
x_tfidf_vect

# min_df = 5 >> It ignores the term that appears in less than 5 documents
# min_df = 0.05 >> It ignores the term that appears in less than 5% documents

,aaron,abandon,abandoned,abilities,ability,able,ably,aboard,abound,abruptly,...,youth,yup,zane,zany,zellweger,zero,zeta,zombie,zone,zooms
0,0.0,0.0,0.0,0.0,0.000000,0.0,0.078118,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.03594,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1996,0.0,0.0,0.0,0.0,0.041443,0.0,0.000000,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1997,0.0,0.0,0.0,0.0,0.065398,0.0,0.000000,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1998,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Train Test Split

In [98]:
x_train, x_test, y_train, y_test = train_test_split(x_tfidf_vect, y, test_size=0.2, random_state=10, stratify=y)

# Model Training

## 1. GaussianNB

In [99]:
gnb_model = GaussianNB()
gnb_model.fit(x_train,y_train)

y_pred = gnb_model.predict(x_test)

acc_score = accuracy_score(y_test, y_pred)
print("Accuracy score for GNB model is :",acc_score)

cnf_matrix = confusion_matrix(y_test, y_pred)
print('Confusion Matrix :\n',cnf_matrix)

clf_report = classification_report(y_test, y_pred)
print('Classification report :\n',clf_report)

Accuracy score for GNB model is : 0.68
Confusion Matrix :
 [[144  56]
 [ 72 128]]
Classification report :
               precision    recall  f1-score   support

           0       0.67      0.72      0.69       200
           1       0.70      0.64      0.67       200

    accuracy                           0.68       400
   macro avg       0.68      0.68      0.68       400
weighted avg       0.68      0.68      0.68       400



## 2. MultinomialNB

In [100]:
mnb_model = MultinomialNB()
mnb_model.fit(x_train,y_train)

y_pred = mnb_model.predict(x_test)

acc_score = accuracy_score(y_test, y_pred)
print("Accuracy score for GNB model is :",acc_score)

cnf_matrix = confusion_matrix(y_test, y_pred)
print('Confusion Matrix :\n',cnf_matrix)

clf_report = classification_report(y_test, y_pred)
print('Classification report :\n',clf_report)

Accuracy score for GNB model is : 0.7875
Confusion Matrix :
 [[165  35]
 [ 50 150]]
Classification report :
               precision    recall  f1-score   support

           0       0.77      0.82      0.80       200
           1       0.81      0.75      0.78       200

    accuracy                           0.79       400
   macro avg       0.79      0.79      0.79       400
weighted avg       0.79      0.79      0.79       400



## 3. BernoulliNB

In [101]:
bnb_model = BernoulliNB()
bnb_model.fit(x_train,y_train)

y_pred = bnb_model.predict(x_test)

acc_score = accuracy_score(y_test, y_pred)
print("Accuracy score for GNB model is :",acc_score)

cnf_matrix = confusion_matrix(y_test, y_pred)
print('Confusion Matrix :\n',cnf_matrix)

clf_report = classification_report(y_test, y_pred)
print('Classification report :\n',clf_report)

Accuracy score for GNB model is : 0.775
Confusion Matrix :
 [[171  29]
 [ 61 139]]
Classification report :
               precision    recall  f1-score   support

           0       0.74      0.85      0.79       200
           1       0.83      0.69      0.76       200

    accuracy                           0.78       400
   macro avg       0.78      0.77      0.77       400
weighted avg       0.78      0.78      0.77       400



In [103]:
from sklearn.neighbors import KNeighborsClassifier

knn_model = KNeighborsClassifier()
knn_model.fit(x_train,y_train)

y_pred = knn_model.predict(x_test)

acc_score = accuracy_score(y_test, y_pred)
print("Accuracy score for GNB model is :",acc_score)

cnf_matrix = confusion_matrix(y_test, y_pred)
print('Confusion Matrix :\n',cnf_matrix)

clf_report = classification_report(y_test, y_pred)
print('Classification report :\n',clf_report)

Accuracy score for GNB model is : 0.6675
Confusion Matrix :
 [[129  71]
 [ 62 138]]
Classification report :
               precision    recall  f1-score   support

           0       0.68      0.65      0.66       200
           1       0.66      0.69      0.67       200

    accuracy                           0.67       400
   macro avg       0.67      0.67      0.67       400
weighted avg       0.67      0.67      0.67       400



In [104]:
# SVM

In [ ]:
credit remark
declined reason:
    reason1: Data entry error : wrong aadhar name
    reason2: declined due to mismatch of date of birth in aadhar and pan
    reason3: decline due to mismatch of date of birth in aadhar and pan
    reason4: error due Data entry person
Lemmatiztion:
        

In [118]:
# print(column_names)

In [134]:

def get_pos_tag(word):
    tag = pos_tag([word])[0][1][0].lower()
    return tag

get_pos_tag('from')
# def lemmatize_text(text):
#     lemma = WordNetLemmatizer()
#     text_list = []
#     tokens = word_tokenize(text)
#     for word in tokens:
#         word = lemma.lemmatize(word,pos = get_pos_tag(word))
#         text_list.append(word)
#     text = ' '.join(text_list)
#     print(text)
#     return text

'i'